In [1]:
# First, install the necessary library
!pip install youtube_transcript_api


### METODE 1: YOUTUBE TRANSKRIP API TO TXT

In [2]:
from youtube_transcript_api import YouTubeTranscriptApi
import re

# Extract video ID from the YouTube URL
def get_video_id(url):
    video_id_match = re.search(r'(?:v=|\/)([0-9A-Za-z_-]{11}).*', url)
    if video_id_match:
        return video_id_match.group(1)
    return None

# URL of the YouTube video
video_url = "https://www.youtube.com/shorts/-1CWqhnL504"
video_id = get_video_id(video_url)

if video_id:
    try:
        # Get the transcript
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
        
        # Format the transcript
        formatted_transcript = ""
        for line in transcript:
            formatted_transcript += f"{line['text']} "
        
        print("Transcript:")
        print(formatted_transcript)
        
        # Save to file
        with open('video_transcript.txt', 'w', encoding='utf-8') as file:
            file.write(formatted_transcript)
        print("\nTranscript saved to 'video_transcript.txt'")
        
    except Exception as e:
        print(f"Error retrieving transcript: {e}")
        print("Note: Not all videos have available transcripts.")
else:
    print("Could not extract video ID from the URL")

Transcript:
i tolerate all ideologies the nationalist ideologies the religious theologies the communist idol so far they are not making trouble or damage to the indonesian state 

Transcript saved to 'video_transcript.txt'


### METHOD 2: OPEN AI WHISPER

In [3]:
# Install required packages
!pip install openai-whisper
!pip install ffmpeg-python



     ---------------------------------------- 0.0/800.5 kB ? eta -:--:--
     ------------------------ ------------- 524.3/800.5 kB 3.4 MB/s eta 0:00:01
     -------------------------------------- 800.5/800.5 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   ---------------------------------------- 894.9/894.9 kB 6.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803437 sha256=9b848f1a2d1f7ea1a6e4c0c23c6de9a8d4a52742f7e0637d6b01a05aed8f3484
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\7c\f5\6f\92094c35416f9397abb86b23cfe72fb255a3013

In [4]:
# Install required packages
!pip install openai-whisper ffmpeg-python pytube

In [6]:
!pip install pytube

In [5]:
import whisper
import os
import re
from pytube import YouTube
import datetime

def get_video_id(url):
    """Extract video ID from YouTube URL"""
    video_id_match = re.search(r'(?:v=|\/)([0-9A-Za-z_-]{11}).*', url)
    if video_id_match:
        return video_id_match.group(1)
    return None

def download_youtube_audio(url, output_path="audio"):
    """Download audio from YouTube video"""
    try:
        # Create output directory if it doesn't exist
        if not os.path.exists(output_path):
            os.makedirs(output_path)
            
        # Create YouTube object
        yt = YouTube(url)
        
        # Get video details for file naming
        video_id = get_video_id(url)
        video_title = yt.title
        safe_title = re.sub(r'[^\w\-_]', '_', video_title)
        
        print(f"Downloading audio from: {video_title}")
        
        # Get audio stream and download
        audio_stream = yt.streams.filter(only_audio=True).first()
        output_file = audio_stream.download(output_path=output_path)
        
        # Rename to mp3
        base, ext = os.path.splitext(output_file)
        new_file = f"{base}.mp3"
        os.rename(output_file, new_file)
        
        print(f"Audio downloaded to: {new_file}")
        return new_file, safe_title
        
    except Exception as e:
        print(f"Error downloading YouTube audio: {e}")
        return None, None

def transcribe_audio(audio_path, language="id"):
    """Transcribe audio using OpenAI Whisper"""
    try:
        print(f"Loading Whisper model...")
        model = whisper.load_model("medium")  # You can change model size: tiny, base, small, medium, large
        
        print(f"Transcribing audio... (this may take some time)")
        result = model.transcribe(audio_path, language=language)
        
        return result["text"]
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None

# Main process
def process_youtube_speech(youtube_url, language="id"):
    """Process YouTube video: download audio and transcribe"""
    # Download audio
    audio_file, video_title = download_youtube_audio(youtube_url)
    
    if not audio_file:
        return
    
    # Transcribe audio
    transcription = transcribe_audio(audio_file, language=language)
    
    if not transcription:
        return
    
    # Save transcription to file
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"Soekarno_{timestamp}.txt"
    
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(f"Source: {youtube_url}\n")
        f.write(f"Title: {video_title}\n")
        f.write(f"Transcribed: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write(transcription)
    
    print(f"✓ Transcription complete! Saved to {output_filename}")
    print("\nPreview:")
    print(transcription[:500] + "..." if len(transcription) > 500 else transcription)
    
    return output_filename

# Example usage
youtube_url = "https://www.youtube.com/watch?v=N_Cp4KBrRZw"  # Change to your Soekarno speech video
process_youtube_speech(youtube_url, language="id")

Error downloading YouTube audio: HTTP Error 400: Bad Request


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

# URL target
url = "https://www.goodreads.com/author/quotes/639030.Mohammad_Hatta"

# User-Agent header untuk meniru browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

quotes_list = []

while True:
    # Mengirim permintaan HTTP
    response = requests.get(url, headers=headers)
    
    # Memeriksa status response
    if response.status_code != 200:
        print(f"Gagal mengakses halaman. Kode status: {response.status_code}")
        break
        
    # Parsing konten HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Mencari semua elemen kutipan
    quotes = soup.find_all('div', class_='quoteText')
    
    # Mengekstrak teks kutipan
    for quote in quotes:
        text = quote.get_text(strip=True).split('―')[0]
        text = text.replace('“', '').replace('”', '').strip()
        quotes_list.append(text)
    
    # Mencari link halaman berikutnya
    next_page = soup.find('a', class_='next_page')
    
    if not next_page:
        break  # Tidak ada halaman berikutnya
        
    url = "https://www.goodreads.com" + next_page['href']
    sleep(2)  # Jeda untuk menghormati server

# Membuat DataFrame
df = pd.DataFrame(quotes_list, columns=['Quotes'])

# Menyimpan ke CSV
df.to_csv('quotes_hatta.csv', index=False)

# Menampilkan hasil
print(f"Berhasil mengumpulkan {len(df)} kutipan:")
df.head()

Berhasil mengumpulkan 17 kutipan:


,Quotes
0,"I’d volunteer to go to prison, as long as ther..."
1,"Aku rela di penjara asalkan bersama buku, kare..."
2,Hanya ada satu negara yang pantas menjadi nega...
3,Filosofi meluaskan pandangan serta mempertajam...
4,Demokrasi hanya berjalan kalau disertai rasa t...
